In [1]:
import cv2
import pdb
import os
import argparse as arg
import time
import numpy as np


In [2]:
from dis import dis
from tqdm import tqdm
from importlib.resources import Package
from skimage.morphology import medial_axis


In [4]:
from function import *
import params as pm
import table

function : There is already a table to find crack width. 8


In [5]:
PATH = '/home/jovyan/DragonBall/vision'
IMG_PATH = PATH+'/data/'
SAVE_DIR = PATH+'/results/'

In [6]:
FL = 4.8
WD = 0.45
SIAH = 14.0
SRPH = 9248.0

In [18]:
file_name = "RP_10_1.jpg"
img_name = file_name[:-4]

distance = (img_name[-2:]) 
if distance == '10':
    distance = 10 * 1000
else :
    distance = int(distance[1]) * 1000
print(distance)

1000


In [19]:
lens = pm.Lens(distance)  # type: ignore

SPP(0.39): 0.12936213991769546 ,  HFOV(1572): 523.9166666666666 , PMAG(0.004): 0.012


In [23]:
img_path = '{0}{1}'.format(IMG_PATH, file_name)
each_save_dir = '{0}{1}/'.format(SAVE_DIR, img_name)
print(img_path, each_save_dir)

/home/jovyan/DragonBall/vision/data/RP_10_1.jpg /home/jovyan/DragonBall/vision/results/RP_10_1/


In [24]:
if not os.path.exists(each_save_dir):
    os.makedirs(each_save_dir)  # type: ignore
    print("made directory")

In [25]:
img = cv2.imread(img_path)
cv2.imwrite(each_save_dir + '{0}_00_original.jpg'.format(img_name), img)

img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.imwrite(each_save_dir + '{0}_01_0_grayscale.jpg'.format(img_name), img)

#2. closing 연산 적용하기 : 검은 색 노이즈 제거
img = closing_func(img)
cv2.imwrite(each_save_dir + '{0}_02_0_closing.jpg'.format(img_name), img)

#3-1. 가우시안 블러 적용 기존 : (7,7) -> (3,3)
filter_size = 5
img = cv2.GaussianBlur(img, (filter_size, filter_size), 0)

#3-2 오츄 알고리즘 적용 : 임계값 T로 이진화
img = otsu_func(img)
cv2.imwrite(each_save_dir + '{0}_03_0_otsu.jpg'.format(img_name), img)

# img = erode_func(img)
# cv2.imwrite(each_save_dir + '{0}_03_1_erode.jpg'.format(img_name), img)

#4. 자잘한 노이즈 제거
img = noise_reduction_func(img, threshold=230)
img[0, :] = img[-1, :] = img[:, 0] = img[:, -1] = 0
cv2.imwrite(each_save_dir + '{0}_04_0_noise.jpg'.format(img_name), img)

#5. 큰 덩어리의 노이즈 제거
img = circle_noise_removal_using_packing_density_func(img)
cv2.imwrite(each_save_dir + '{0}_05_circle_noise_.jpg'.format(img_name), img)



True

In [26]:

#6. 세선화 과정
img_thinned = thinning_func(img)

#7-1. 분기점(특징점)을 주변 색의 편차를 보고 찾음
start_interval_point_direction_key_list = search_start_interval_point_direction_key(img_thinned)
# print("+++++++++++++",start_interval_point_direction_key_list[0:5])

#7-2. 특징점을 탐색하여, 선분 세그먼트의 시작점부터 끝점을 찾음
total_segment_list, total_chain_list = search_edge_segment(img_thinned, start_interval_point_direction_key_list)

#7-3. 선분 세그먼트의 시작점부터 끝점까지의 길이를 진행한 방향을 통해 계산
total_length_list = crack_length_func(total_chain_list)
#img = img_thinned

#7-4. 균열 선분 이미지에 분기점 시각화하기
color = pm.Color()
img_thinBGR = cv2.cvtColor(img_thinned, cv2.COLOR_GRAY2BGR)
img_length_interval_point = color.display_crack_color(img_thinBGR, total_segment_list, total_length_list, mode='interval_point')
cv2.imwrite(each_save_dir + '{0}_7_img_length_interval_point.jpg'.format(img_name), img_length_interval_point)


100%|██████████| 716/716 [00:00<00:00, 1555.95it/s]


True

In [34]:
from function import *
class Color:
    def __init__(self):
        (minN, midN, maxN) = (0, 150, 255)
        self.WIDEST_COLOR_G = [maxN, minN, maxN] #Red########333333333333333333333333333[minN, maxN, minN] #Green
        self.WIDER_COLOR_Y  = [midN, midN, maxN] #Yellow##########3333333333333333333333
        self.NORMAL_COLOR_R = [midN, midN, midN] #Gray[midN, minN, maxN] #Red########333333333333333333333333333
        self.LOWER_COLOR_B  = [maxN, minN, minN] #Blue
        self.LOWEST_COLOR_G = [midN, midN, midN] #Gray
        self.ZERO_COLOR_B = [minN, minN, minN] #Gray
        self.first_lowest = self.second = self.third = self.fourth_highest = 0

    def dividing_3(self, total_LoW_list, mode='width'):
        if mode == 'width':
            sorted_set_list = list(set.union(*map(set, total_LoW_list)))
        else:     
            sorted_set_list = list(set(total_LoW_list).union())

        num_divided_by_3 = round(len(sorted_set_list) / 3)
        # print(len(sorted_set_list),sorted_set_list)
        num_4th_highest = sorted_set_list[-1]
        num_3th = sorted_set_list[-1 * num_divided_by_3]
        num_2nd = sorted_set_list[-2 * num_divided_by_3]
        num_1st_lowest = sorted_set_list[0]

        return num_1st_lowest, num_2nd, num_3th, num_4th_highest

    def display_crack_color(self, img_thin, segment_list, pixel_list, mode='width', direction_key_list=[[0,0, 'O']]):
        
        if mode == 'interval_point':
            for i, j, _ in direction_key_list:
                cv2.circle(img_thin, (j, i), 10, (0, 255, 0), thickness=3)
        
        num_1st_lowest, num_2nd, num_3th, num_4th_highest = self.dividing_3(total_LoW_list=pixel_list, mode=mode)
        #num_1st_lowest, num_2nd, num_3th, num_4th_highest = (0, 4, 5, 60)

        total_segment_zip = zip(segment_list, pixel_list)

        for segment_block, crack in total_segment_zip:
            if mode == 'width':
                each_segment_block = zip(segment_block, crack)
                for [row, col], crack_width in each_segment_block:
                    if num_1st_lowest <= crack_width < num_2nd:
                        img_thin[row, col] = self.NORMAL_COLOR_R
                    elif num_2nd <= crack_width < num_3th:
                        img_thin[row, col] = self.WIDER_COLOR_Y
                    elif num_3th <= crack_width <= num_4th_highest:
                        img_thin[row, col] = self.WIDEST_COLOR_G
            else :
                for [row, col] in segment_block:
                    if num_1st_lowest <= crack < num_2nd:
                        img_thin[row, col] = self.NORMAL_COLOR_R
                    elif num_2nd <= crack < num_3th:
                        img_thin[row, col] = self.WIDER_COLOR_Y
                    elif num_3th <= crack <= num_4th_highest:
                        img_thin[row, col] = self.WIDEST_COLOR_G

        return img_thin
'''    def fill_crack_color_width(self, img, chain_block, pixel_list):
 
        num_1st_lowest, num_2nd, num_3th, num_4th_highest = (0, 4, 5, 60)
        total_segment_zip = zip(chain_block, pixel_list)

        for chain_block, crack in total_segment_zip:
            each_segment_block = zip(chain_block, crack)
            for [row, col], crack_width in each_segment_block:
                if num_1st_lowest <= crack_width < num_2nd:
                    match_color = self.NORMAL_COLOR_R
                elif num_2nd <= crack_width < num_3th:
                    match_color = self.WIDER_COLOR_Y
                elif num_3th <= crack_width <= num_4th_highest:
                    match_color = self.WIDEST_COLOR_G
                else : match_color = self.LOWEST_COLOR_G
                    
                img[row, col] = match_color

        return img'''

color = Color()

# 추가된 너비 측정 방식 0 - new one
def renewal_profiling_crack_width_func(img, img_th, total_segment_list, total_chain_list):
    total_width_list = []
    print("Dd")
    for j in range(len(total_chain_list)):
        segment_block = total_chain_list[j]
        center_pixel_block = total_segment_list[j]
        segment_width_block = []
        for i in range(0, len(segment_block)):
            start = center_pixel_block[i]
            if i == 0 : direc = segment_block[1]
            else :      direc = segment_block[i]
            # left : 진행 방향 기준 왼쪽 수직 방향 
            left_width  = finding_white_until_black(img, img_th, start, direc, LorR=0)
            # right : 진행 방향 기준 오른쪽 수직 방향 
            right_width = finding_white_until_black(img, img_th, start, direc, LorR=1)
            
            segment_width_block.append(left_width+right_width)
            
        #
        total_width_list.append(segment_width_block)
    #
    return total_width_list, img

width_func = 'profiling _1'
#        8. 선분의 너비를 구하기 위한 함수
#        8-1. 폭을 계산한 리스트 만들기(모든 균열 포인트마다)
#        default : adaptive
#        usable width func : normal, adaptive, profiling_1 profiling_2
if width_func == 'adaptive':
    total_width_list = adaptive_crack_width_func(img, total_segment_list)
elif width_func == 'normal':
    total_width_list = normal_crack_width_func(img, total_segment_list, radius=7)
elif width_func == 'profiling_1':
    total_width_list, img_pro = profiling_crack_width_func(img, img_thinned, total_segment_list, total_chain_list)
else :
    total_width_list, img_pro = renewal_profiling_crack_width_func(img, img_thinned, total_segment_list, total_chain_list)

print(len(total_segment_list), len(total_chain_list), len(total_width_list))

for i in range(len(total_segment_list)):
    if len(total_segment_list[i]) != len(total_width_list[i]):
        print(len(total_chain_list[i]), len(total_width_list[i]))
        print((total_segment_list[i]), total_chain_list[i], (total_width_list[i]))
        break


#        8-2. 균열 선분 이미지에 폭을 색상으로 시각화하기
img_width = color.display_crack_color(img_thin=img_thinBGR, segment_list=total_segment_list, pixel_list=total_width_list, mode='width')
cv2.imwrite(each_save_dir + '{0}_8_mask_width_visualization.jpg'.format(img_name), img_width)


Dd
357 357 357


True

In [33]:
#        8-2. 균열 이미지에 폭을 색상으로 시각화하기
img_width = fill_crack_width_func(img=img, img_th=img_thinned, total_segment_list=total_segment_list, total_chain_list=total_chain_list, total_width_list=total_width_list)
cv2.imwrite(each_save_dir + '{0}_9_full_width_visualization.jpg'.format(img_name), img_width)

True